In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import os
from tqdm import tqdm
import pickle
from copy import deepcopy
import uuid
from urllib.request import urlretrieve
from zipfile import ZipFile

In [2]:
"""
Uncomment the below code if you are using google colab.
"""

# !git clone https://github.com/shubhamgantayat/Traffic-Signs-Detection-Testing.git
# %cd Traffic-Signs-Detection-Testing/notebook/

Cloning into 'Traffic-Signs-Detection-Testing'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 38 (delta 0), reused 2 (delta 0), pack-reused 34
Unpacking objects: 100% (38/38), done.
/content/Traffic-Signs-Detection-Testing/notebook


In [4]:
ZIP_TRAIN_PATH = "../raw_data/train.pickle.zip"
ZIP_VALID_PATH = "../raw_data/valid.pickle.zip"
ZIP_TEST_PATH = "../raw_data/test.pickle.zip"

In [5]:
def unzip_file(src, dest):
    with ZipFile(src, "r") as f:
        f.extractall(dest)

In [6]:
unzip_file(ZIP_TRAIN_PATH, "../raw_data/")
unzip_file(ZIP_VALID_PATH, "../raw_data/")
unzip_file(ZIP_TEST_PATH, "../raw_data/")

In [7]:
TRAIN_PATH = "../raw_data/train.pickle"
VALID_PATH = "../raw_data/valid.pickle"
TEST_PATH = "../raw_data/test.pickle"

In [8]:
def load_file(path):
    with open(path, 'rb') as f:
        d = pickle.load(f, encoding='latin1')  
    x = d['features'].astype(np.uint8)   # 4D numpy.ndarray type, for train = (34799, 32, 32, 3)
    y = d['labels']                        # 1D numpy.ndarray type, for train = (34799,)
    s = d['sizes']                         # 2D numpy.ndarray type, for train = (34799, 2)
    c = d['coords']  
    return x, y, s, c


In [9]:
train_data, valid_data, test_data = {}, {}, {}
train_data["x"], train_data["y"], _, train_data["c"] = load_file(TRAIN_PATH)
valid_data["x"], valid_data["y"], _, valid_data["c"] = load_file(VALID_PATH)
test_data["x"], test_data["y"], _, test_data["c"] = load_file(TEST_PATH)

In [10]:
def display(img, coordinates):
    new_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.rectangle(new_img, 
                  (coordinates[0], coordinates[1]),
                  (coordinates[0] + coordinates[2], coordinates[1] + coordinates[3]),
                  (255,0,0),
                  1
                 )
    while True:
        cv2.imshow("img", new_img)
        if cv2.waitKey(0) & 0xFF == 27:
            cv2.destroyAllWindows()
            break

In [11]:
# display(train_data["x"][4000], train_data["c"][4000])

In [12]:
labels = pd.read_csv("../raw_data/label_names.csv")

In [13]:
labels

,ClassId,SignName
0,0,Speed limit (20km/h)
1,1,Speed limit (30km/h)
2,2,Speed limit (50km/h)
3,3,Speed limit (60km/h)
4,4,Speed limit (70km/h)
5,5,Speed limit (80km/h)
6,6,End of speed limit (80km/h)
7,7,Speed limit (100km/h)
8,8,Speed limit (120km/h)
9,9,No passing


# Augmentation 1 - Local Histogram Equalization

In [14]:
def local_histogram_equalization(img):
    new_img = np.zeros(img.shape, dtype=np.uint8)
    new_img[:,:,0] = cv2.equalizeHist(img[:,:,0])
    new_img[:,:,1] = cv2.equalizeHist(img[:,:,1])
    new_img[:,:,2] = cv2.equalizeHist(img[:,:,2])
    return new_img

In [15]:
new_img = local_histogram_equalization(train_data['x'][4000])

In [16]:
# display(new_img, train_data['c'][4000])

# Augmentation 2 - Changing Brightness

In [17]:
def change_brightness(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    img_hsv[:,:,2] = img_hsv[:,:,2] * (0.5 + np.random.uniform(size=(img_hsv.shape[:-1])))
    img_rgb = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
    return img_rgb

In [18]:
new_img = change_brightness(train_data['x'][4000])

In [19]:
# display(new_img, train_data['c'][4000])

# Augmentation 3 - Rotating Image

In [20]:
def rotation_changing(image):
    # Defining angle range
    angle_range = 30
    # Defining angle rotation
    angle_rotation = np.random.uniform(angle_range) - angle_range / 2
    # Getting shape of image
    rows, columns, channels = image.shape
    # Implementing rotation
    # Calculating Affine Matrix
    affine_matrix = cv2.getRotationMatrix2D((columns / 2, rows / 2), angle_rotation, 1)
    # Warping original image with Affine Matrix
    rotated_image = cv2.warpAffine(image, affine_matrix, (columns, rows))
    # Returning rotated image
    return rotated_image

In [21]:
new_img = rotation_changing(train_data['x'][4000])

In [22]:
# display(new_img, train_data['c'][4000])

# Preprocessing Data

In [23]:
def shuffle(data, seed=0):
    new_data = deepcopy(data)
    np.random.seed(seed)
    np.random.shuffle(new_data["x"])
    np.random.seed(seed)
    np.random.shuffle(new_data["y"])
    np.random.seed(seed)
    np.random.shuffle(new_data["c"])
    return new_data

In [24]:
def preprocess(data, shuffle=False, lhe=False, rotate=False, brightness=False):
    if shuffle:
        data = shuffle(data)
    if lhe:
        data["x"] = list(map(local_histogram_equalization, tqdm(data["x"])))
    if rotate:
        data["x"] = list(map(rotation_changing, tqdm(data["x"])))
    if brightness:
        data["x"] = list(map(rotation_changing, tqdm(data["x"])))
    return data

In [25]:
def join_data(data, augmented_data):
    data["x"] = np.r_[data["x"], augmented_data["x"]]
    data["y"] = np.r_[data["y"], augmented_data["y"]]
    data["c"] = np.r_[data["c"], augmented_data["c"]]
    return data

In [26]:
def generate_augmented_data(data):
    augmented_data = None
    list_kwargs = [
        {"shuffle": False, "lhe": True, "rotate": False, "brightness": False},
        {"shuffle": False, "lhe": False, "rotate": True, "brightness": False},
        {"shuffle": False, "lhe": False, "rotate": False, "brightness": True},
        {"shuffle": False, "lhe": True, "rotate": True, "brightness": False},
        {"shuffle": False, "lhe": True, "rotate": False, "brightness": True},
        {"shuffle": False, "lhe": False, "rotate": True, "brightness": True},
        {"shuffle": False, "lhe": True, "rotate": True, "brightness": True}
    ]
    for kwargs in list_kwargs:
        data = shuffle(data, np.random.randint(0,100))
        data_top = {}
        data_top["x"] = data["x"][:int(0.2*len(data["x"]))]
        data_top["y"] = data["y"][:int(0.2*len(data["x"]))]
        data_top["c"] = data["c"][:int(0.2*len(data["x"]))]
        new_data = preprocess(data_top, **kwargs)
        if augmented_data is None:
            augmented_data = new_data
        else:
            augmented_data = join_data(new_data, augmented_data)
    return join_data(new_data, augmented_data)

In [27]:
augmented_train_data = generate_augmented_data(train_data)
augmented_valid_data = generate_augmented_data(valid_data)

100%|██████████| 882/882 [00:00<00:00, 42518.63it/s]


# Labelling Of Data

In [28]:
def convert_coordinates_to_yolo_annotation(coordinates, label, shape):
    dh, dw, _ = shape
    x,y,w,h = coordinates
    x_final = x + w if x + w < dw else dw
    y_final = y + h if y + h < dh else dh
    x_mean = (x + x_final) / (dw * 2)
    y_mean = (y + y_final) / (dh * 2)
    w_norm = (x_final - x) / dw
    h_norm = (y_final - y) / dh
    return [" ".join([str(label), str(x_mean), str(y_mean), str(w_norm), str(h_norm)])]

def convert_yolo_annotations_to_coordinates(annotation, shape):
    dh, dw, _ = shape
    _, x, y, w, h = [float(i) for i in annotation[0].strip().split()]
    x_start = (x - w / 2) * dw
    y_start = (y - h / 2) * dh
    return np.array([x_start, y_start, w * dw, h * dh], dtype=np.uint8)

In [29]:
annotation = convert_coordinates_to_yolo_annotation(train_data['c'][4000],
                                       train_data['y'][4000],
                                       train_data['x'][4000].shape
                                      )

In [30]:
new_coords = convert_yolo_annotations_to_coordinates(annotation, train_data['x'][4000].shape)

In [31]:
# display(train_data['x'][4000] ,new_coords)

# Saving new data to file 

In [32]:
def create_directory_structure(root):
    try:
        os.makedirs(root, exist_ok=True)
        folders = ["train", "valid", "test"]
        for folder in folders:
            path_to_folder = root / folder
            os.makedirs(path_to_folder, exist_ok=True)
            os.makedirs(path_to_folder / "images", exist_ok=True)
            os.makedirs(path_to_folder / "labels", exist_ok=True)
        return True
    except:
        return False

In [33]:
ROOT_DIR = Path("../data/")
create_directory_structure(ROOT_DIR)

True

In [34]:
def save_images_and_labels(path, img, label, coordinates):
    try:
        name = str(uuid.uuid1())
        img_name = name + ".jpg"
        label_name = name + ".txt"
        img_path = path / "images" / img_name
        label_path = path / "labels" / label_name
        cv2.imwrite(str(img_path), img)
        with open(label_path, "w") as f:
            f.writelines(convert_coordinates_to_yolo_annotation(coordinates, label, img.shape))
        return True
    except Exception as e:
        return str(e)

In [35]:
TRAIN_FOLDER = ROOT_DIR / "train"
VALID_FOLDER = ROOT_DIR / "valid"
TEST_FOLDER = ROOT_DIR / "test"
train_result = list(map(lambda img, label, coordinates:save_images_and_labels(TRAIN_FOLDER, img, label, coordinates), tqdm(augmented_train_data["x"]), augmented_train_data["y"], augmented_train_data["c"]))   
valid_result = list(map(lambda img, label, coordinates:save_images_and_labels(VALID_FOLDER, img, label, coordinates), tqdm(augmented_valid_data["x"]), augmented_valid_data["y"], augmented_valid_data["c"]))    
test_result = list(map(lambda img, label, coordinates:save_images_and_labels(TEST_FOLDER, img, label, coordinates), tqdm(test_data["x"]), test_data["y"], test_data["c"]))                           

 22%|██▏       | 2792/12630 [00:01<00:04, 2252.44it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in ubyte_scalars
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in ubyte_scalars
  
100%|██████████| 12630/12630 [00:05<00:00, 2309.66it/s]


In [36]:
count = 0
for i in test_result:
    if i == False:
        count += 1
print(count)

0


In [37]:
nc = len(labels)

In [38]:
nc

43

In [39]:
classes = str(list(labels["SignName"]))

In [40]:
desc = f"""train: ../data/train/images
val: ../data/valid/images
test: ../data/test/images

nc: {nc}
names: {classes}
"""

with open("../data/data.yaml", "w") as f:
    f.write(desc)

# Setting up yolov5

In [1]:
"""
Uncomment if you are using google colab
"""
# %cd ../

'\nUncomment if you are using google colab\n'

In [42]:
pwd

'/content/Traffic-Signs-Detection-Testing'

In [43]:
!git clone https://github.com/ultralytics/yolov5.git  # clone repo
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 13033, done.
remote: Total 13033 (delta 0), reused 0 (delta 0), pack-reused 13033
Receiving objects: 100% (13033/13033), 11.91 MiB | 24.79 MiB/s, done.
Resolving deltas: 100% (9059/9059), done.
/content/Traffic-Signs-Detection-Testing/yolov5


In [44]:
pwd

'/content/Traffic-Signs-Detection-Testing/yolov5'

In [45]:
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
# from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 596 kB 5.4 MB/s 
Setup complete. Using torch 1.11.0+cu113 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


In [46]:
%cat ../data/data.yaml

train: ../data/train/images
val: ../data/valid/images
test: ../data/test/images

nc: 43
names: ['Speed limit (20km/h)', 'Speed limit (30km/h)', 'Speed limit (50km/h)', 'Speed limit (60km/h)', 'Speed limit (70km/h)', 'Speed limit (80km/h)', 'End of speed limit (80km/h)', 'Speed limit (100km/h)', 'Speed limit (120km/h)', 'No passing', 'No passing for vehicles over 3.5 metric tons', 'Right-of-way at the next intersection', 'Priority road', 'Yield', 'Stop', 'No vehicles', 'Vehicles over 3.5 metric tons prohibited', 'No entry', 'General caution', 'Dangerous curve to the left', 'Dangerous curve to the right', 'Double curve', 'Bumpy road', 'Slippery road', 'Road narrows on the right', 'Road work', 'Traffic signals', 'Pedestrians', 'Children crossing', 'Bicycles crossing', 'Beware of ice/snow', 'Wild animals crossing', 'End of all speed and passing limits', 'Turn right ahead', 'Turn left ahead', 'Ahead only', 'Go straight or right', 'Go straight or left', 'Keep right', 'Keep left', 'Roundabout

In [47]:
import yaml
with open("../data/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [48]:
num_classes

'43'

In [49]:
%cat ../yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [50]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [51]:
%%writetemplate models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
!python train.py --img 416 --batch 16 --epochs 20 --data ../data/data.yaml --cfg ./models/custom_yolov5s.yaml --weights yolov5s.pt --name yolov5s_results

/content/Traffic-Signs-Detection-Testing/yolov5
train: weights=yolov5s.pt, cfg=./models/custom_yolov5s.yaml, data=../data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-163-gb53917d torch 1.11.0+cu113 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, i

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/Traffic-Signs-Detection-Testing/yolov5/runs/train/yolov5s_results/results.png', width=1000)

In [ ]:
print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/Traffic-Signs-Detection-Testing/yolov5/runs/train/yolov5s_results/val_batch0_labels.jpg', width=900)

In [ ]:
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/Traffic-Signs-Detection-Testing/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

In [ ]:
%ls runs/

In [ ]:
%ls runs/train/yolov5s_results/weights

In [ ]:
%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source ../data/test/images

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp2*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Saving the weights to drive

In [ ]:
import time
while True:
  pass

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cp /content/yolov5/runs/train/yolov5s_results/weights/best.pt /content/gdrive/My%20Projects/Traffic-Signs-Testing/